In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define namespaces
namespaces = {
    'head': 'urn:iso:std:iso:20022:tech:xsd:head.001.001.03',
    'isct': 'urn:iso:std:iso:20022:tech:xsd:pacs.009.001.09'
}

# Load and parse the XML file
tree = ET.parse("pacs.009.001.09.xml")
root = tree.getroot()

# Function to extract data from a pacs.009 message
def extract_pacs009_data(root):
    return {
        # Header info
        'sender_bic': root.findtext('.//head:Fr/head:FIId/head:FinInstnId/head:BICFI', namespaces=namespaces),
        'receiver_bic': root.findtext('.//head:To/head:FIId/head:FinInstnId/head:BICFI', namespaces=namespaces),
        'biz_msg_id': root.findtext('.//head:BizMsgIdr', namespaces=namespaces),
        'msg_def_id': root.findtext('.//head:MsgDefIdr', namespaces=namespaces),
        'creation_dt': root.findtext('.//head:CreDt', namespaces=namespaces),

        # Group header
        'message_id': root.findtext('.//isct:GrpHdr/isct:MsgId', namespaces=namespaces),
        'message_creation_dt': root.findtext('.//isct:GrpHdr/isct:CreDtTm', namespaces=namespaces),
        'settlement_method': root.findtext('.//isct:GrpHdr/isct:SttlmInf/isct:SttlmMtd', namespaces=namespaces),
        'instructing_agent_bic': root.findtext('.//isct:GrpHdr/isct:InstgAgt/isct:FinInstnId/isct:BICFI', namespaces=namespaces),
        'instructed_agent_bic': root.findtext('.//isct:GrpHdr/isct:InstdAgt/isct:FinInstnId/isct:BICFI', namespaces=namespaces),

        # Credit transfer info
        'instr_id': root.findtext('.//isct:CdtTrfTxInf/isct:PmtId/isct:InstrId', namespaces=namespaces),
        'end_to_end_id': root.findtext('.//isct:CdtTrfTxInf/isct:PmtId/isct:EndToEndId', namespaces=namespaces),
        'tx_id': root.findtext('.//isct:CdtTrfTxInf/isct:PmtId/isct:TxId', namespaces=namespaces),
        'amount': root.findtext('.//isct:CdtTrfTxInf/isct:IntrBkSttlmAmt', namespaces=namespaces),
        'currency': root.find('.//isct:CdtTrfTxInf/isct:IntrBkSttlmAmt', namespaces=namespaces).attrib.get('Ccy'),
        'settlement_date': root.findtext('.//isct:CdtTrfTxInf/isct:IntrBkSttlmDt', namespaces=namespaces)
    }

# Extract and convert to DataFrame
data = extract_pacs009_data(root)
df = pd.DataFrame([data])
df


,sender_bic,receiver_bic,biz_msg_id,msg_def_id,creation_dt,message_id,message_creation_dt,settlement_method,instructing_agent_bic,instructed_agent_bic,instr_id,end_to_end_id,tx_id,amount,currency,settlement_date
0,{bicfi},ISSDINTL07X,{BizMsgIdr},pacs.009.001.09,{CreDt},{MsgId},{CreDtTm},{interbank_settlement_method},{bicfi},{bicfi},{InstrId},{EndToEndId},{TxId},{IntrBkSttlmAmt},{Ccy},{IntrBkSttlmDt}
